### Parse Products

This PySpark notebook parses product information from a simulated business case. Product information is stored in a blob storage account which is provided by the user. We will simply retrieve the data, display a portion of the data, and gather basic descriptive statistics on the dataset.

#### Enter your blob storage account in the next cell

In [ ]:
blob_storage_name = '<Blob Storage Account>'
public_parameters_container = "publicparameters"

#### Import modules and create the Spark context
The following modules are used to run the SparkSession and gather the descriptive statistics.

The Spark context sets up internal services and establishes a connection to a Spark execution environment. A Spark context acts as a client of Spark’s execution environment.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pandas as pd

spark = SparkSession.builder.enableHiveSupport().getOrCreate()
## in spark 2.0: you can access the Spark context through spark.sparkContext, and access the spark through spark
sc = spark.sparkContext

#### Define functions

The following functions will be used to help parse the data read from our blob storage account

In [ ]:
## eliminate the double quote in the headers
def eliminate_double_quote(header):
    return [s.replace('"','') for s in header]
## convert numeric fields from string in each line of RDD
def convert_string_to_numeric_df(p,index):
    return [str(pi).replace('"','') if i in index else float(pi) for i,pi in enumerate(p)]
## return the index of one string vector in another string vector
def columnindex(header,fields):
    return [header.index(s) for s in fields]
## construct schema of the data frame
def construct_schema(fields_categorical, header):
    return StructType([StructField(field_name, StringType(), True) if field_name in fields_categorical else StructField(field_name, DoubleType(), True) for field_name in header])
## define the stores in treament, control and other group

#### Import file from blob storage

The following cell will access the blob using the storage account information provided in the first cell

In [ ]:
## import file from blob storage
products_d_loc = "wasb://{1}@{0}.blob.core.windows.net/".format(blob_storage_name,public_parameters_container)+"products.csv"
products_d_file = sc.textFile(products_d_loc)

#### Define functions

The following cell will read, parse, and output product information.

In [ ]:
## define categorical fields and numerical fields
# products data
header_products_original=eliminate_double_quote([str(s) for s in products_d_file.first().split(',')])
header_products=['department_id', 'brand_id', 'product_id', 'msrp', 'cost']
fields_categorical_products=["department_id","brand_id","product_id"]
fields_numerical_products=list(set(header_products)-set(fields_categorical_products))

## turn data into Data Frame
# products data
columnindex_categorical_products=columnindex(header_products,fields_categorical_products)
schema_products=construct_schema(fields_categorical_products, header_products)
df_products_rdd=products_d_file.filter(lambda l: header_products_original[0] not in l).map(lambda p: convert_string_to_numeric_df(p.split(","),columnindex_categorical_products))
df_products=spark.createDataFrame(df_products_rdd,schema_products)

pd_products = df_products.toPandas()

# Show top 20 rows
df_products.show()

#### Descriptive statistics

The following cell will provide basic descriptive statistics 

In [ ]:
#Describe products
pd_products.describe()